<a href="https://colab.research.google.com/github/Djensonsan/Information_Retrieval_Assignment_2/blob/main/LDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Information Retrieval Assignment 2: LDA


## Runtime specs

In [45]:
!cat /proc/cpuinfo

processor	: 0
vendor_id	: GenuineIntel
cpu family	: 6
model		: 79
model name	: Intel(R) Xeon(R) CPU @ 2.20GHz
stepping	: 0
microcode	: 0x1
cpu MHz		: 2199.998
cache size	: 56320 KB
physical id	: 0
siblings	: 2
core id		: 0
cpu cores	: 1
apicid		: 0
initial apicid	: 0
fpu		: yes
fpu_exception	: yes
cpuid level	: 13
wp		: yes
flags		: fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush mmx fxsr sse sse2 ss ht syscall nx pdpe1gb rdtscp lm constant_tsc rep_good nopl xtopology nonstop_tsc cpuid tsc_known_freq pni pclmulqdq ssse3 fma cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt aes xsave avx f16c rdrand hypervisor lahf_lm abm 3dnowprefetch invpcid_single ssbd ibrs ibpb stibp fsgsbase tsc_adjust bmi1 hle avx2 smep bmi2 erms invpcid rtm rdseed adx smap xsaveopt arat md_clear arch_capabilities
bugs		: cpu_meltdown spectre_v1 spectre_v2 spec_store_bypass l1tf mds swapgs taa
bogomips	: 4399.99
clflush size	: 64
cache_alignment	: 64
address sizes	: 46 bits physical, 48 b

In [46]:
!cat /proc/meminfo

MemTotal:       13333596 kB
MemFree:         9682140 kB
MemAvailable:   12097800 kB
Buffers:           83416 kB
Cached:          2327872 kB
SwapCached:            0 kB
Active:          1066100 kB
Inactive:        2305268 kB
Active(anon):     785944 kB
Inactive(anon):      340 kB
Active(file):     280156 kB
Inactive(file):  2304928 kB
Unevictable:           0 kB
Mlocked:               0 kB
SwapTotal:             0 kB
SwapFree:              0 kB
Dirty:               640 kB
Writeback:             0 kB
AnonPages:        960104 kB
Mapped:           288928 kB
Shmem:               940 kB
Slab:             182244 kB
SReclaimable:     142720 kB
SUnreclaim:        39524 kB
KernelStack:        4592 kB
PageTables:         7760 kB
NFS_Unstable:          0 kB
Bounce:                0 kB
WritebackTmp:          0 kB
CommitLimit:     6666796 kB
Committed_AS:    3721568 kB
VmallocTotal:   34359738367 kB
VmallocUsed:           0 kB
VmallocChunk:          0 kB
Percpu:              952 kB
AnonHugePages:   

## Imports

In [47]:
# Install your required packages here
!pip install pandas numpy matplotlib fsspec gcsfs dask[complete] gensim nltk
!pip install -q tqdm

In [48]:
import pandas as pd
import numpy as np
import tqdm.notebook as tqdm
import dask.dataframe as dd

from ast import literal_eval
import gensim
from gensim import corpora, models
from tqdm import tqdm
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
from nltk.corpus import stopwords
import nltk
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [49]:
# Mount google drive in colab:
from google.cloud import storage
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Preprocessing

In [50]:
# Pycharm:
# data = pd.read_csv('data/news_dataset.csv')

In [51]:
# Colab:
data = pd.read_csv('/content/drive/MyDrive/IR-Assignment-2/data/NewsArticles.csv', encoding='mac_roman')

### Exploration

In [52]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3824 entries, 0 to 3823
Columns: 144 entries, article_id to Unnamed: 143
dtypes: float64(4), int64(1), object(139)
memory usage: 4.2+ MB


In [53]:
data.head(n=43)

,article_id,publish_date,article_source_link,title,subtitle,text,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31,Unnamed: 32,Unnamed: 33,Unnamed: 34,Unnamed: 35,Unnamed: 36,Unnamed: 37,Unnamed: 38,Unnamed: 39,...,Unnamed: 104,Unnamed: 105,Unnamed: 106,Unnamed: 107,Unnamed: 108,Unnamed: 109,Unnamed: 110,Unnamed: 111,Unnamed: 112,Unnamed: 113,Unnamed: 114,Unnamed: 115,Unnamed: 116,Unnamed: 117,Unnamed: 118,Unnamed: 119,Unnamed: 120,Unnamed: 121,Unnamed: 122,Unnamed: 123,Unnamed: 124,Unnamed: 125,Unnamed: 126,Unnamed: 127,Unnamed: 128,Unnamed: 129,Unnamed: 130,Unnamed: 131,Unnamed: 132,Unnamed: 133,Unnamed: 134,Unnamed: 135,Unnamed: 136,Unnamed: 137,Unnamed: 138,Unnamed: 139,Unnamed: 140,Unnamed: 141,Unnamed: 142,Unnamed: 143
0,1,2017/2/7,http://abcnews.go.com/Politics/pence-break-tie...,"Betsy DeVos Confirmed as Education Secretary, ...",NaN,Michigan billionaire education activist Betsy ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,2017/2/7,http://abcnews.go.com/Politics/wireStory/melan...,Melania Trump Says White House Could Mean Mill...,NaN,First lady Melania Trump has said little about...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,2017/2/7,http://abcnews.go.com/Politics/wireStory/trump...,"As Trump Fears Fraud, GOP Eliminates Election ...",NaN,A House committee voted on Tuesday to eliminat...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,2017/2/7,http://abcnews.go.com/Politics/appeals-court-d...,Appeals Court to Decide on Challenge to Trump'...,NaN,"This afternoon, three federal judges from the ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,2017/2/7,http://abcnews.go.com/US/23-states-winter-weat...,At Least 4 Tornadoes Reported in Southeast Lou...,NaN,At least four tornadoes touched down in Louisi...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,6,2017/2/7,http://abcnews.go.com/International/wireStory/...,Mother of Backpacker Slain in Australia Critic...,NaN,The mother of a backpacker slain in an Austral...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,7,2017/2/7,http://abcnews.go.com/Politics/trumps-labor-se...,Trump's Labor Secretary Pick Andrew Puzder Adm...,NaN,"Donald Trump's pick for labor secretary, Andre...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,Na

In [54]:
# Use document 42 as running example
data.loc[42, 'text']

'Wheelchair-bound and barely able to talk in his later years, the South Africa legend passed away Monday aged 45. Motor neurone disease had left him a shadow of the gladiator who helped the Springboks win the World Cup in 1995, but his contribution to the game will never be forgotten, says former teammate Cobus Visagie. "Joost basically created the new requirements for a modern No. 9," the former South Africa front-row prop told CNN\'s World Sport show Tuesday. Van der Westhuizen was 6 foot 2 inches tall, but played at scrumhalf -- a position traditionally occupied by the game\'s smallest players, providing the link between teams\' backs and forwards. "Some days I looked down when we were standing in a circle and saw a man with boots three sizes bigger than myself -- a scrumhalf!" added Visagie, who played alongside van der Westhuizen at the 1999 World Cup and won 29 caps for his country. "He had incredible speed, he had incredible bravery -- he\'s very well known for the tackles he ma

### Keep document content

In [55]:
data_content = data['text']

In [56]:
type(data_content)

pandas.core.series.Series

In [57]:
data_content.head(n=3)

0    Michigan billionaire education activist Betsy ...
1    First lady Melania Trump has said little about...
2    A House committee voted on Tuesday to eliminat...
Name: text, dtype: object

### Tokenization, Stemming and Lemmatization

In [58]:
tqdm.pandas()
# Note Jens: Might want to use Dask to speed things up.
# When using Dask can't use tqdm as far as I know.

In [59]:
# There's NaN values in the dataset
data_content.dropna(inplace=True)

In [60]:
data_content.isna().any()

False

In [61]:
# Tokenization
data_content_tokenized = data_content.progress_apply(lambda x: nltk.word_tokenize(x))

100%|██████████| 3791/3791 [00:16<00:00, 227.94it/s]


In [62]:
# Remove words smaller than 3 characters
data_content_tokenized = data_content_tokenized.progress_apply(lambda x: [y for y in x if len(y)>2])

100%|██████████| 3791/3791 [00:00<00:00, 12226.64it/s]


In [63]:
# Stemming and Lemmatization
stemmer = SnowballStemmer("english")
data_content_stemmed = data_content_tokenized.progress_apply(lambda x: [stemmer.stem(WordNetLemmatizer().lemmatize(y)) for y in x])

100%|██████████| 3791/3791 [00:29<00:00, 127.36it/s]


In [64]:
# Remove Stopswords
stop_words = set(stopwords.words('english'))
data_content_clean = data_content_stemmed.progress_apply(lambda x: [y for y in x if not y in stop_words])

100%|██████████| 3791/3791 [00:00<00:00, 12290.54it/s]


In [65]:
data_content_clean.head()

0    [michigan, billionair, educ, activist, betsi, ...
1    [first, ladi, melania, trump, ha, said, littl,...
2    [hous, committe, vote, tuesday, elimin, indepe...
3    [afternoon, three, feder, judg, 9th, circuit, ...
4    [least, four, tornado, touch, louisiana, today...
Name: text, dtype: object

In [66]:
# data_content_clean contain the cleaned 'content' column of the news dataset:
data_content_clean.to_csv('/content/drive/MyDrive/IR-Assignment-2/data/new_dataset_clean.csv')

# LDA

The following part will use Python libraries for performing an LDA. The result can be used for comparison with our model.

The code follows a tutorial:
[LDA Tutorial](https://towardsdatascience.com/topic-modeling-and-latent-dirichlet-allocation-in-python-9bf156893c24)

In [67]:
data_content_clean = pd.read_csv('/content/drive/MyDrive/IR-Assignment-2/data/new_dataset_clean.csv')

In [68]:
data_content_clean = data_content_clean['text']

In [69]:
data_content_clean.head()

0    ['michigan', 'billionair', 'educ', 'activist',...
1    ['first', 'ladi', 'melania', 'trump', 'ha', 's...
2    ['hous', 'committe', 'vote', 'tuesday', 'elimi...
3    ['afternoon', 'three', 'feder', 'judg', '9th',...
4    ['least', 'four', 'tornado', 'touch', 'louisia...
Name: text, dtype: object

In [70]:
dictionary = gensim.corpora.Dictionary()
for article in tqdm(data_content_clean):
  dictionary.add_documents([literal_eval(article)])

100%|██████████| 3791/3791 [00:03<00:00, 1051.10it/s]


## Sanity check

In [71]:
print(dictionary)

Dictionary(51736 unique tokens: ['12-11', '14,000', '24-hour', '50-50', '51-50']...)


In [72]:
count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break

0 12-11
1 14,000
2 24-hour
3 50-50
4 51-50
5 abc
6 above-averag
7 account
8 activist
9 addit
10 administr


In [73]:
# Filter out tokens that appear in less than 15 documents, more than 0.5 documents (fraction of total corpus size, not absolute number).
# keep only the first 100000 most frequent tokens.
dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)

## doc2bow

For each document we create a dictionary reporting how many words and how many times those words appear.

In [74]:
bow_corpus = [dictionary.doc2bow(literal_eval(doc)) for doc in data_content_clean]
bow_corpus[42]

[(3, 1),
 (15, 2),
 (68, 1),
 (77, 3),
 (119, 1),
 (126, 1),
 (155, 1),
 (168, 2),
 (201, 1),
 (281, 1),
 (285, 1),
 (307, 2),
 (338, 1),
 (366, 1),
 (374, 1),
 (401, 1),
 (419, 1),
 (444, 3),
 (536, 1),
 (573, 2),
 (630, 1),
 (635, 1),
 (730, 1),
 (750, 1),
 (755, 1),
 (771, 1),
 (773, 1),
 (807, 1),
 (865, 1),
 (895, 1),
 (901, 1),
 (910, 2),
 (937, 1),
 (998, 1),
 (1120, 1),
 (1133, 1),
 (1215, 1),
 (1216, 1),
 (1226, 1),
 (1250, 1),
 (1278, 1),
 (1289, 1),
 (1292, 1),
 (1298, 1),
 (1318, 1),
 (1357, 2),
 (1432, 1),
 (1471, 1),
 (1504, 1),
 (1567, 1),
 (1629, 1),
 (1847, 1),
 (1974, 1),
 (2043, 1),
 (2212, 1),
 (2278, 1),
 (2405, 1),
 (2406, 1),
 (2407, 1),
 (2408, 1),
 (2409, 1),
 (2410, 1),
 (2411, 1),
 (2412, 1),
 (2413, 1),
 (2414, 1),
 (2415, 1),
 (2416, 1),
 (2417, 1),
 (2418, 1),
 (2419, 1),
 (2420, 1),
 (2421, 1),
 (2422, 1),
 (2423, 1),
 (2424, 1),
 (2425, 1),
 (2426, 2),
 (2427, 1)]

In [75]:
bow_doc_42 = bow_corpus[42]
for i in range(len(bow_doc_42)):
    print("Word {} (\"{}\") appears {} time.".format(bow_doc_42[i][0],
                                               dictionary[bow_doc_42[i][0]],
bow_doc_42[i][1]))

Word 3 ("account") appears 1 time.
Word 15 ("believ") appears 2 time.
Word 68 ("imagin") appears 1 time.
Word 77 ("like") appears 3 time.
Word 119 ("say") appears 1 time.
Word 126 ("sinc") appears 1 time.
Word 155 ("work") appears 1 time.
Word 168 ("ani") appears 2 time.
Word 201 ("could") appears 1 time.
Word 281 ("money") appears 1 time.
Word 285 ("n't") appears 1 time.
Word 307 ("possibl") appears 2 time.
Word 338 ("sold") appears 1 time.
Word 366 ("well") appears 1 time.
Word 374 ("2008") appears 1 time.
Word 401 ("dead") appears 1 time.
Word 419 ("go") appears 1 time.
Word 444 ("look") appears 3 time.
Word 536 ("consid") appears 1 time.
Word 573 ("matter") appears 2 time.
Word 630 ("way") appears 1 time.
Word 635 ("activ") appears 1 time.
Word 730 ("dozen") appears 1 time.
Word 750 ("list") appears 1 time.
Word 755 ("open") appears 1 time.
Word 771 ("wound") appears 1 time.
Word 773 ("admit") appears 1 time.
Word 807 ("appear") appears 1 time.
Word 865 ("war") appears 1 time.
Word

## TF-IDF

In [76]:
tfidf = models.TfidfModel(bow_corpus)
corpus_tfidf = tfidf[bow_corpus]
for doc in corpus_tfidf:
    print(doc)
    break

[(0, 0.096725172741217), (1, 0.096725172741217), (2, 0.050668384095748406), (3, 0.03931924473585671), (4, 0.05544892779083032), (5, 0.08370963685179755), (6, 0.057316685510420205), (7, 0.057303231728936775), (8, 0.037105249157914515), (9, 0.06775828560672804), (10, 0.04930115291599818), (11, 0.06905895052969102), (12, 0.046918258494267125), (13, 0.026514811865213178), (14, 0.06020365231635402), (15, 0.030670169763667397), (16, 0.07914502301238711), (17, 0.04847024198465567), (18, 0.049771592264296644), (19, 0.04612641263297113), (20, 0.0646384766387566), (21, 0.059655317071140364), (22, 0.019524160182389748), (23, 0.06427038709375343), (24, 0.0795767389614727), (25, 0.07369399040895541), (26, 0.08191134943205915), (27, 0.0352785780953645), (28, 0.05469567127693268), (29, 0.07108831082485668), (30, 0.09708833710796923), (31, 0.1716254074422707), (32, 0.039102054270830626), (33, 0.0342349731862914), (34, 0.03129416613180567), (35, 0.187276609810157), (36, 0.0456177627200195), (37, 0.0775

## LDA Library: gensim.models.LdaMulticore

In [77]:
lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=20, id2word=dictionary, passes=2, workers=2)

In [78]:
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.025*"china" + 0.012*"compani" + 0.011*"say" + 0.010*"chines" + 0.009*"market" + 0.007*"percent" + 0.007*"invest" + 0.007*"develop" + 0.006*"busi" + 0.006*"countri"
Topic: 1 
Words: 0.008*"peopl" + 0.007*"say" + 0.005*"food" + 0.005*"time" + 0.005*"use" + 0.005*"n't" + 0.004*"world" + 0.004*"mani" + 0.004*"make" + 0.004*"onli"
Topic: 2 
Words: 0.021*"polic" + 0.010*"offic" + 0.008*"man" + 0.008*"woman" + 0.008*"two" + 0.007*"kill" + 0.007*"investig" + 0.006*"attack" + 0.006*"arrest" + 0.006*"prison"
Topic: 3 
Words: 0.012*"patient" + 0.011*"hospit" + 0.010*"health" + 0.009*"care" + 0.008*"say" + 0.008*"doctor" + 0.007*"work" + 0.007*"drug" + 0.007*"medic" + 0.006*"peopl"
Topic: 4 
Words: 0.032*"attack" + 0.013*"kill" + 0.011*"target" + 0.011*"peopl" + 0.008*"pakistan" + 0.007*"wound" + 0.007*"countri" + 0.007*"two" + 0.006*"refuge" + 0.005*"2015"
Topic: 5 
Words: 0.012*"would" + 0.011*"govern" + 0.011*"brexit" + 0.010*"minist" + 0.010*"may" + 0.009*"ireland" + 0.009*"

## Evaluation

In [79]:
# Let's look at a test document: document 42
# 1) clearly see that the article is about the economy 40%
# 2) US politics 27%
# 3) US Justice Department 8%
# 4) US Congress 6%
for index, score in sorted(lda_model[bow_corpus[42]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model.print_topic(index, 10)))


Score: 0.6352262496948242	 
Topic: 0.008*"peopl" + 0.007*"say" + 0.007*"like" + 0.007*"n't" + 0.006*"time" + 0.006*"trump" + 0.005*"want" + 0.005*"new" + 0.005*"woman" + 0.005*"get"

Score: 0.14167258143424988	 
Topic: 0.025*"china" + 0.012*"compani" + 0.011*"say" + 0.010*"chines" + 0.009*"market" + 0.007*"percent" + 0.007*"invest" + 0.007*"develop" + 0.006*"busi" + 0.006*"countri"

Score: 0.133882075548172	 
Topic: 0.006*"child" + 0.006*"peopl" + 0.006*"school" + 0.006*"water" + 0.005*"would" + 0.005*"home" + 0.005*"n't" + 0.005*"day" + 0.004*"say" + 0.004*"famili"

Score: 0.08042789250612259	 
Topic: 0.008*"peopl" + 0.007*"say" + 0.005*"food" + 0.005*"time" + 0.005*"use" + 0.005*"n't" + 0.004*"world" + 0.004*"mani" + 0.004*"make" + 0.004*"onli"


In [80]:
# Decide the most representative documents for each topic and write it to file

repList = [[] for i in range(20)]

for doc_id in tqdm(range(len(data_content)), "Scoring: "):
  for index, score in sorted(lda_model[bow_corpus[doc_id]], key=lambda tup: -1*tup[1]):
    repList[index].append((doc_id, score))

for topic in repList:
  topic.sort(key=lambda x:x[1], reverse=True)

topicList = ["Topic "+str(i) for i in range(20)]

df = pd.DataFrame(columns=topicList)

for topic in range(len(topicList)):
  end = 100
  if len(repList[topic]) < 100:
    end = len(repList[topic])-1
  df["Topic "+ str(topic)] = [x[0] for x in repList[topic][0:end]]

df.to_csv('/content/drive/MyDrive/IR-Assignment-2/data/topic_document_rank_library.csv')


Scoring: 100%|██████████| 3791/3791 [00:04<00:00, 789.62it/s]


In [81]:
file = open("/content/drive/MyDrive/IR-Assignment-2/data/analysis_topic_per_doc_lib.txt","w") 

for i in range(10):
  file.write("doc " + str(i) + " topics: \n")
  for index, score in sorted(lda_model[bow_corpus[i]], key=lambda tup: -1*tup[1]):
    file.write(str(score) + "   ")
  file.write("\n\n")

file.close()